# Using huggingface's transformers
(containg pretrained models fot auto text sumerization)

## Import libraries

In [6]:
from transformers import pipeline
import nltk
nltk.download('punkt')
nltk.download('stopwords')
import re
import torch
from transformers import BartTokenizer, BartForConditionalGeneration, BartConfig
BART_PATH = 'facebook/bart-large-cnn'
from rouge import Rouge

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\makye\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\makye\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
# function to open file
def openFile(file):
    with open (file, 'r') as bk:
        text = bk.read()
        return text
    
#Calling function on file
my_book = openFile('chapterone.txt')


In [8]:
def cleanText(text):
    text = re.sub(r'\n|\r', ' ', text)
    text = re.sub(r' +', ' ', text)
    text = text.strip()
    return text

doc = cleanText(my_book)

In [9]:
def nest_sentences(document):
    nested = []
    sent = []
    length = 0
    for sentence in nltk.sent_tokenize(document):
        length += len(sentence)
        if length < 1024:
            sent.append(sentence)
        else:
            nested.append(sent)
            sent = []
            length = 0

    if sent:
        nested.append(sent)

    return nested

In [11]:
nested = nest_sentences(doc)
len(nested)
# nested[13]

189

In [12]:
%%time
bart_model = BartForConditionalGeneration.from_pretrained(BART_PATH, output_past=True)
bart_tokenizer = BartTokenizer.from_pretrained(BART_PATH, output_past=True)

Wall time: 20.9 s


In [13]:
def generate_summary(nested_sentences):
  device = 'cuda'
  summaries = []
  for nested in nested_sentences:
    input_tokenized = bart_tokenizer.encode(' '.join(nested), truncation=True, return_tensors='pt')
    input_tokenized = input_tokenized.to(device)
    summary_ids = bart_model.to('cuda').generate(input_tokenized,
                                      length_penalty=3.0,
                                      min_length=30,
                                      max_length=100)
    output = [bart_tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]
    summaries.append(output)
  summaries = [sentence for sublist in summaries for sentence in sublist]
  return summaries

In [14]:
%%time
summ = generate_summary(nested)

Wall time: 4min 1s


In [15]:
summ

['Mr. Phileas Fogg lived, in 1872, at No. 7, Saville Row, Burlington Gardens. He was one of the most noticeable members of the Reform Club. People said that he resembled Byron.',
 'Phileas Fogg was a member of the Reform, and that was all. He was recommended by the Barings, with whom he had an open credit. His cheques were regularly paid at sight from his account current.',
 'Phileas Fogg was the least communicative of men. He talked very little, and seemed all the more mysterious for his taciturn manner. His daily habits were quite open to observation. But whatever he did was so exactly the same thing that he had always done before.',
 'Phileas Fogg was not known to have either wife or children. He lived alone in his house in Saville Row, whither none penetrated. He breakfasted and dined at the club, at hours mathematically fixed.',
 'He passed ten hours out of the twenty-four in Saville Row, either in sleeping or making his toilet. When he chose to take a walk it was with a regular s

## using pretrained models/pipeline for auto-text summerization

In [16]:
nested_summ = nest_sentences(' '.join(summ))
nested_summ

[['Mr. Phileas Fogg lived, in 1872, at No.',
  '7, Saville Row, Burlington Gardens.',
  'He was one of the most noticeable members of the Reform Club.',
  'People said that he resembled Byron.',
  'Phileas Fogg was a member of the Reform, and that was all.',
  'He was recommended by the Barings, with whom he had an open credit.',
  'His cheques were regularly paid at sight from his account current.',
  'Phileas Fogg was the least communicative of men.',
  'He talked very little, and seemed all the more mysterious for his taciturn manner.',
  'His daily habits were quite open to observation.',
  'But whatever he did was so exactly the same thing that he had always done before.',
  'Phileas Fogg was not known to have either wife or children.',
  'He lived alone in his house in Saville Row, whither none penetrated.',
  'He breakfasted and dined at the club, at hours mathematically fixed.',
  'He passed ten hours out of the twenty-four in Saville Row, either in sleeping or making his toile

In [17]:
%%time
model_output = generate_summary(nested_summ)

Wall time: 59.1 s


In [18]:
model_output

['Mr. Phileas Fogg lived, in 1872, at No. 7, Saville Row, Burlington Gardens. He was one of the most noticeable members of the Reform Club. People said that he resembled Byron.',
 'Phileas Fogg was awaiting his successor, who was due at the house between eleven and half-past. He had dismissed James Forster because that luckless youth had brought him shaving-water at eighty-four degrees Fahrenheit instead of eighty-six.',
 'At first some rash individuals espoused his cause, which became still more popular when the Illustrated London News came out with his portrait. But five days after the article in the Geographical Society appeared, the demand began to subside. Lord Albemarle, an elderly paralytic gentleman, was now the only advocate of Phileas Fogg left.',
 'The passenger bowed to Fix, and returned to the steamer. Fix felt sure that Phileas Fogg would not land at Suez, but was really going on to Bombay. "Consul," said he, "I have no longer any doubt. I have spotted my man"',
 '"Quite 

In [19]:
# reference source: https://www.britannica.com/topic/Around-the-World-in-Eighty-Days-by-Verne
# Import refrence summary
ref_file = openFile('ref.txt')

#Clean refrence summary
ref_summary = cleanText(ref_file)

In [20]:
def Convert(string):
    li = list(string.split("."))
    return li

In [21]:
reference = Convert(ref_summary)
reference

['Phileas Fogg, a London gentleman of meticulous and unchanging habits, hires as his valet Jean Passepartout, a Frenchman who has had a variety of jobs, including circus performer, but now seeks a tranquil life',
 ' After reading in The Daily Telegraph that a new railroad in India has made it theoretically possible to travel around the world in 80 days, Fogg bets his fellow members at the Reform Club that he will make that journey in 80 days or less; the wager is for the princely sum of Â£20,000 (half his fortune)',
 ' Leaving that night, Fogg and a nonplussed Passepartout board a train bound for Dover and Calais to begin their journey',
 ' Shortly before Foggâ€™s departure, someone resembling him had robbed a bank, and Foggâ€™s sudden exit leads Scotland Yard to believe that he was the bank robber',
 ' Accordingly, a detective, Mr',
 ' Fix, is sent to Suez, in British-ruled Egypt, to await the steamer Mongolia, on which Fogg and Passepartout are traveling',
 ' Fix befriends Passpartou

In [22]:
rouge = Rouge()

In [23]:
rouge.get_scores(model_output, reference, avg=True, ignore_empty=True)

{'rouge-1': {'f': 0.10215427346256739,
  'p': 0.08342842844826588,
  'r': 0.1709300478149504},
 'rouge-2': {'f': 0.003190357471492217,
  'p': 0.003228659885006634,
  'r': 0.0035014005602240893},
 'rouge-l': {'f': 0.1012663192936988,
  'p': 0.08211062131402522,
  'r': 0.16368965157701232}}